In [1]:
from __future__ import print_function, division

from keras.datasets import mnist, cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os

import numpy as np

Using TensorFlow backend.


<h2>DCGAN</h2>

In [2]:
class DCGAN():
    def __init__(self):

        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        self.generator = self.build_generator()

        z = Input(shape=(100,))
        img = self.generator(z)

        self.discriminator.trainable = False

        valid = self.discriminator(img)

        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        (X_train, _), (_, _) = cifar10.load_data()

        X_train = X_train / 127.5 - 1.

        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            g_loss = self.combined.train_on_batch(noise, valid)

            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,:])
                axs[i,j].axis('off')
                cnt += 1
        
        if not os.path.isdir('images/'):
            os.makedirs('images/')

        fig.savefig("images/cifar10_%d.png" % epoch)
        plt.close()

In [ ]:
dcgan = DCGAN()
dcgan.train(epochs=4000, batch_size=32, save_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 9, 9, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 9, 9, 64)          0         
__________

/home/padilha/anaconda3/envs/python2/lib/python2.7/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.156301, acc.: 32.81%] [G loss: 0.817357]
1 [D loss: 0.668570, acc.: 62.50%] [G loss: 1.269707]
2 [D loss: 0.651260, acc.: 62.50%] [G loss: 1.609016]
3 [D loss: 0.793230, acc.: 54.69%] [G loss: 1.791023]
4 [D loss: 0.899128, acc.: 40.62%] [G loss: 1.591161]
5 [D loss: 0.867331, acc.: 54.69%] [G loss: 1.336334]
6 [D loss: 0.821526, acc.: 39.06%] [G loss: 1.455488]
7 [D loss: 0.776087, acc.: 54.69%] [G loss: 1.341943]
8 [D loss: 0.703241, acc.: 60.94%] [G loss: 1.537189]
9 [D loss: 0.705008, acc.: 54.69%] [G loss: 1.569446]
10 [D loss: 0.704202, acc.: 54.69%] [G loss: 1.551774]
11 [D loss: 0.555903, acc.: 68.75%] [G loss: 1.417365]
12 [D loss: 0.726356, acc.: 54.69%] [G loss: 1.150061]
13 [D loss: 0.496291, acc.: 75.00%] [G loss: 1.097916]
14 [D loss: 0.605069, acc.: 70.31%] [G loss: 0.599423]
15 [D loss: 0.498453, acc.: 85.94%] [G loss: 0.351528]
16 [D loss: 0.429611, acc.: 76.56%] [G loss: 0.554495]
17 [D loss: 0.314906, acc.: 90.62%] [G loss: 0.599780]
18 [D loss: 0.375422

149 [D loss: 0.862283, acc.: 59.38%] [G loss: 0.993232]
150 [D loss: 1.175389, acc.: 35.94%] [G loss: 1.131015]
151 [D loss: 0.756980, acc.: 50.00%] [G loss: 1.151168]
152 [D loss: 0.968839, acc.: 42.19%] [G loss: 0.989468]
153 [D loss: 0.599992, acc.: 71.88%] [G loss: 1.252107]
154 [D loss: 0.546028, acc.: 71.88%] [G loss: 1.054676]
155 [D loss: 0.751399, acc.: 54.69%] [G loss: 0.957748]
156 [D loss: 0.804498, acc.: 54.69%] [G loss: 1.146086]
157 [D loss: 1.099767, acc.: 28.12%] [G loss: 1.178723]
158 [D loss: 0.743185, acc.: 62.50%] [G loss: 1.375986]
159 [D loss: 0.714191, acc.: 60.94%] [G loss: 1.017344]
160 [D loss: 0.423313, acc.: 84.38%] [G loss: 1.017166]
161 [D loss: 0.693325, acc.: 64.06%] [G loss: 0.955276]
162 [D loss: 0.484560, acc.: 81.25%] [G loss: 1.097349]
163 [D loss: 0.663913, acc.: 59.38%] [G loss: 1.377809]
164 [D loss: 0.647292, acc.: 53.12%] [G loss: 0.871985]
165 [D loss: 0.662929, acc.: 62.50%] [G loss: 0.859115]
166 [D loss: 0.791539, acc.: 50.00%] [G loss: 1.

296 [D loss: 0.583385, acc.: 67.19%] [G loss: 0.984117]
297 [D loss: 0.921426, acc.: 39.06%] [G loss: 1.146405]
298 [D loss: 0.625338, acc.: 62.50%] [G loss: 0.943500]
299 [D loss: 0.796540, acc.: 59.38%] [G loss: 1.182077]
300 [D loss: 0.651618, acc.: 64.06%] [G loss: 1.083375]
301 [D loss: 0.676663, acc.: 57.81%] [G loss: 1.124183]
302 [D loss: 0.654750, acc.: 62.50%] [G loss: 1.351994]
303 [D loss: 0.817094, acc.: 54.69%] [G loss: 1.253098]
304 [D loss: 0.585438, acc.: 65.62%] [G loss: 1.295189]
305 [D loss: 1.036504, acc.: 40.62%] [G loss: 0.855300]
306 [D loss: 0.724632, acc.: 54.69%] [G loss: 0.851900]
307 [D loss: 0.715503, acc.: 53.12%] [G loss: 0.884578]
308 [D loss: 0.899084, acc.: 45.31%] [G loss: 0.510279]
309 [D loss: 0.607500, acc.: 71.88%] [G loss: 0.981513]
310 [D loss: 0.746690, acc.: 51.56%] [G loss: 0.791220]
311 [D loss: 0.563408, acc.: 70.31%] [G loss: 0.670801]
312 [D loss: 0.739631, acc.: 64.06%] [G loss: 0.502542]
313 [D loss: 0.469761, acc.: 81.25%] [G loss: 0.

...

3334 [D loss: 0.627797, acc.: 64.06%] [G loss: 0.620951]
3335 [D loss: 0.862413, acc.: 31.25%] [G loss: 0.647420]
3336 [D loss: 0.896135, acc.: 34.38%] [G loss: 0.661504]
3337 [D loss: 0.716525, acc.: 53.12%] [G loss: 0.755269]
3338 [D loss: 0.959794, acc.: 25.00%] [G loss: 0.750546]
3339 [D loss: 0.840049, acc.: 40.62%] [G loss: 0.722029]
3340 [D loss: 0.687970, acc.: 57.81%] [G loss: 0.639098]
3341 [D loss: 0.728230, acc.: 60.94%] [G loss: 0.717369]
3342 [D loss: 0.739769, acc.: 54.69%] [G loss: 0.778883]
3343 [D loss: 0.699745, acc.: 50.00%] [G loss: 0.640102]
3344 [D loss: 0.735283, acc.: 53.12%] [G loss: 0.830715]
3345 [D loss: 0.720017, acc.: 54.69%] [G loss: 0.682170]
3346 [D loss: 0.740603, acc.: 51.56%] [G loss: 0.658529]
3347 [D loss: 0.675227, acc.: 56.25%] [G loss: 0.704628]
3348 [D loss: 0.661676, acc.: 64.06%] [G loss: 0.659114]
3349 [D loss: 0.674726, acc.: 59.38%] [G loss: 0.810061]
3350 [D loss: 0.615882, acc.: 73.44%] [G loss: 0.717356]
3351 [D loss: 0.793285, acc.: 4

3478 [D loss: 0.647548, acc.: 57.81%] [G loss: 1.097615]
3479 [D loss: 0.880537, acc.: 31.25%] [G loss: 0.594979]
3480 [D loss: 0.625929, acc.: 67.19%] [G loss: 0.755011]
3481 [D loss: 0.922261, acc.: 32.81%] [G loss: 0.703345]
3482 [D loss: 0.748563, acc.: 53.12%] [G loss: 0.720990]
3483 [D loss: 0.631136, acc.: 62.50%] [G loss: 0.753062]
3484 [D loss: 0.574800, acc.: 71.88%] [G loss: 0.832825]
3485 [D loss: 0.898213, acc.: 28.12%] [G loss: 0.648193]
3486 [D loss: 0.768206, acc.: 42.19%] [G loss: 0.561856]
3487 [D loss: 0.601072, acc.: 71.88%] [G loss: 0.862566]
3488 [D loss: 0.692957, acc.: 59.38%] [G loss: 0.792374]
3489 [D loss: 0.635221, acc.: 62.50%] [G loss: 0.908583]
3490 [D loss: 0.587611, acc.: 71.88%] [G loss: 0.808613]
3491 [D loss: 0.611258, acc.: 64.06%] [G loss: 0.685429]
3492 [D loss: 0.863463, acc.: 31.25%] [G loss: 0.741795]
3493 [D loss: 0.859346, acc.: 42.19%] [G loss: 0.721315]
3494 [D loss: 0.840234, acc.: 46.88%] [G loss: 0.687228]
3495 [D loss: 0.749615, acc.: 4

3622 [D loss: 0.722834, acc.: 50.00%] [G loss: 0.688736]
3623 [D loss: 0.714540, acc.: 53.12%] [G loss: 0.824440]
3624 [D loss: 0.716666, acc.: 48.44%] [G loss: 0.732035]
3625 [D loss: 0.778115, acc.: 39.06%] [G loss: 0.746095]
3626 [D loss: 0.730181, acc.: 51.56%] [G loss: 0.757768]
3627 [D loss: 0.726541, acc.: 53.12%] [G loss: 0.728341]
3628 [D loss: 0.724101, acc.: 43.75%] [G loss: 0.733510]
3629 [D loss: 0.858065, acc.: 32.81%] [G loss: 0.754101]
3630 [D loss: 0.657844, acc.: 59.38%] [G loss: 0.764690]
3631 [D loss: 0.763894, acc.: 51.56%] [G loss: 0.769043]
3632 [D loss: 0.619118, acc.: 65.62%] [G loss: 0.870661]
3633 [D loss: 0.709809, acc.: 50.00%] [G loss: 0.720195]
3634 [D loss: 0.763063, acc.: 56.25%] [G loss: 0.712752]
3635 [D loss: 0.613535, acc.: 65.62%] [G loss: 0.964556]
3636 [D loss: 0.797000, acc.: 50.00%] [G loss: 0.808887]
3637 [D loss: 0.760850, acc.: 43.75%] [G loss: 0.783123]
3638 [D loss: 0.605431, acc.: 70.31%] [G loss: 0.978444]
3639 [D loss: 0.548106, acc.: 7

3766 [D loss: 0.559857, acc.: 70.31%] [G loss: 0.921447]
3767 [D loss: 0.705514, acc.: 56.25%] [G loss: 0.700878]
3768 [D loss: 0.740614, acc.: 42.19%] [G loss: 0.811355]
3769 [D loss: 0.706134, acc.: 56.25%] [G loss: 0.905901]
3770 [D loss: 0.945993, acc.: 40.62%] [G loss: 0.558921]
3771 [D loss: 0.851520, acc.: 25.00%] [G loss: 0.648847]
3772 [D loss: 0.607834, acc.: 70.31%] [G loss: 0.672692]
3773 [D loss: 0.533048, acc.: 78.12%] [G loss: 0.764318]
3774 [D loss: 0.744824, acc.: 51.56%] [G loss: 0.610178]
3775 [D loss: 0.658507, acc.: 57.81%] [G loss: 0.610764]
3776 [D loss: 0.740149, acc.: 48.44%] [G loss: 0.772126]
3777 [D loss: 0.905922, acc.: 17.19%] [G loss: 0.747508]
3778 [D loss: 0.551218, acc.: 70.31%] [G loss: 0.720134]
3779 [D loss: 0.857040, acc.: 31.25%] [G loss: 0.631398]
3780 [D loss: 0.625842, acc.: 59.38%] [G loss: 0.665905]
3781 [D loss: 0.688926, acc.: 48.44%] [G loss: 0.652413]
3782 [D loss: 0.785863, acc.: 45.31%] [G loss: 0.811746]
3783 [D loss: 0.545173, acc.: 7

3910 [D loss: 0.693857, acc.: 53.12%] [G loss: 0.850417]
3911 [D loss: 0.720014, acc.: 53.12%] [G loss: 0.630091]
3912 [D loss: 0.724112, acc.: 53.12%] [G loss: 0.750952]
3913 [D loss: 0.667103, acc.: 60.94%] [G loss: 0.845620]
3914 [D loss: 0.596248, acc.: 71.88%] [G loss: 0.749559]
3915 [D loss: 0.609138, acc.: 68.75%] [G loss: 0.806897]
3916 [D loss: 0.598234, acc.: 62.50%] [G loss: 0.799215]
3917 [D loss: 0.642824, acc.: 64.06%] [G loss: 0.893617]
3918 [D loss: 0.554382, acc.: 78.12%] [G loss: 0.774090]
3919 [D loss: 0.673656, acc.: 56.25%] [G loss: 0.709726]
3920 [D loss: 0.620824, acc.: 68.75%] [G loss: 0.799555]
3921 [D loss: 0.630342, acc.: 60.94%] [G loss: 1.148254]
3922 [D loss: 0.674239, acc.: 56.25%] [G loss: 1.029232]
3923 [D loss: 0.611221, acc.: 62.50%] [G loss: 0.858089]
3924 [D loss: 0.828129, acc.: 39.06%] [G loss: 0.624099]
3925 [D loss: 0.694108, acc.: 57.81%] [G loss: 0.844417]
3926 [D loss: 0.675052, acc.: 59.38%] [G loss: 1.028890]
3927 [D loss: 0.863856, acc.: 2